# Seminar QF: Credit Risk Analysis Pipeline

This notebook implements a comprehensive pipeline for credit risk analysis using various volatility models:
1.  **Merton Model**: To estimate asset values and volatility from equity data.
2.  **GARCH(1,1)**: Single-regime volatility modeling.
3.  **Regime-Switching (Hamilton Filter)**: Volatility modeling with regime changes.
4.  **MS-GARCH**: Optimized Markov-Switching GARCH model.
5.  **Monte Carlo Simulation**: To forecast future asset values.
6.  **CDS Spread Calculation**: Estimating credit default swap spreads based on simulated default probabilities.

### 0. Setup
Initialize the environment, add `src` to the system path, and define configuration.

In [ ]:
# Setup
import sys
import os
import shutil
from pathlib import Path
import pandas as pd
import numpy as np

# Add project root to path so we can import src
# Assuming notebook is in notebooks/, project root is parent
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import config and modules
try:
    from src.utils import config
    from src.data.data_processing import load_and_preprocess_data, run_merton_estimation, load_interest_rates
    from src.models.garch_model import run_garch_estimation
    from src.models.regime_switching import run_regime_switching_estimation
    from src.models.ms_garch_optimized import run_ms_garch_estimation_optimized
    from src.models.probability_of_default import run_pd_pipeline, calculate_merton_pd_normal
    from src.analysis.result_summary import generate_results_summary
    from src.analysis.monte_carlo_garch import monte_carlo_garch_1year_parallel  # Use parallel version
    from src.analysis.volatility_diagnostics import run_volatility_diagnostics, filter_problematic_firms
    from src.analysis.monte_carlo_regime_switching import monte_carlo_regime_switching_1year_parallel  # Use parallel version
    from src.analysis.monte_carlo_ms_garch import monte_carlo_ms_garch_1year_parallel  # Use parallel version
    from src.analysis.cds_spread_calculator import CDSSpreadCalculator
    from src.analysis.cds_correlation import run_cds_correlation_analysis, plot_cds_correlations

    print("Imports successful.")
    print(f"Data Directory: {config.DATA_DIR}")
except ImportError as e:
    print(f"Import Error: {e}")
    # print(f"sys.path: {sys.path}")

Imports successful.
Data Directory: /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data


### 1. Cache Cleanup
Remove intermediate files.

In [2]:
# Cache Cleanup
print("Cleaning up cache files...")
cache_dir = config.INTERMEDIATES_DIR
cache_files = [
    'merton_results_cache.pkl',
    'mc_garch_cache.csv'
]

for cache_file in cache_files:
    cache_path = cache_dir / cache_file
    if cache_path.exists():
        try:
            os.remove(cache_path)
            print(f"✓ Deleted: {cache_path}")
        except Exception as e:
            print(f"⚠ Could not delete {cache_path}: {e}")
    else:
        print(f"  (No cached file: {cache_path})")

print("Cache cleanup complete.")


Cleaning up cache files...
✓ Deleted: /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/intermediates/merton_results_cache.pkl
  (No cached file: /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/intermediates/mc_garch_cache.csv)
Cache cleanup complete.


### 2. Data Loading & Merton Model Estimation
*   **Load Interest Rates**: From ECB data.
*   **Load Equity Data**: From Excel inputs.
*   **Run Merton Model**: Solves for Asset Value ($V_t$) and Asset Volatility ($\sigma_A$) using the iterative approach.
*   **Output**: `merged_data_with_merton.csv` and `daily_asset_returns.csv`.

In [3]:
# Load Interest Rates
interest_rates_df = load_interest_rates()
print(f"Loaded {len(interest_rates_df)} months of interest rate data")

# Load Equity/Liability Data
df = load_and_preprocess_data()

# Run Merton Model
df_merged, daily_returns_df = run_merton_estimation(df, interest_rates_df)

# Save Results
df_merged.to_csv(config.OUTPUT_DIR / "merged_data_with_merton.csv", index=False)
daily_returns_df.to_csv(config.OUTPUT_DIR / "daily_asset_returns.csv", index=False)

print(f"Saved to {config.OUTPUT_DIR}")

Loaded 384 months of interest rate data
Loading equity data...
Removed 16 flagged companies (gvkeys: [101248, 25466, 203053, 245663, 19349, 243774, 17828, 333645, 101305, 61214, 15181, 14140, 100312, 101276, 100737, 214881])
Remaining firms: 34

Loading liability data...
Removed 16 flagged companies (gvkeys: [101248, 25466, 203053, 245663, 19349, 243774, 17828, 333645, 101305, 61214, 15181, 14140, 100312, 101276, 100737, 214881])
Remaining firms: 34

Loading liability data...
Loaded liability data

MERTON MODEL ESTIMATION (Vectorized + Parallelized - EXACT ndtr)

Processing 34 firms with -1 parallel jobs...

Starting parallel Merton estimation...
Loaded liability data

MERTON MODEL ESTIMATION (Vectorized + Parallelized - EXACT ndtr)

Processing 34 firms with -1 parallel jobs...

Starting parallel Merton estimation...


Loaded 384 months of interest rate data
Loading equity data...
Removed 16 flagged companies (gvkeys: [101248, 25466, 203053, 245663, 19349, 243774, 17828, 333645, 101305, 61214, 15181, 14140, 100312, 101276, 100737, 214881])
Remaining firms: 34

Loading liability data...
Removed 16 flagged companies (gvkeys: [101248, 25466, 203053, 245663, 19349, 243774, 17828, 333645, 101305, 61214, 15181, 14140, 100312, 101276, 100737, 214881])
Remaining firms: 34

Loading liability data...
Loaded liability data

MERTON MODEL ESTIMATION (Vectorized + Parallelized - EXACT ndtr)

Processing 34 firms with -1 parallel jobs...

Starting parallel Merton estimation...
Loaded liability data

MERTON MODEL ESTIMATION (Vectorized + Parallelized - EXACT ndtr)

Processing 34 firms with -1 parallel jobs...

Starting parallel Merton estimation...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Loaded 384 months of interest rate data
Loading equity data...
Removed 16 flagged companies (gvkeys: [101248, 25466, 203053, 245663, 19349, 243774, 17828, 333645, 101305, 61214, 15181, 14140, 100312, 101276, 100737, 214881])
Remaining firms: 34

Loading liability data...
Removed 16 flagged companies (gvkeys: [101248, 25466, 203053, 245663, 19349, 243774, 17828, 333645, 101305, 61214, 15181, 14140, 100312, 101276, 100737, 214881])
Remaining firms: 34

Loading liability data...
Loaded liability data

MERTON MODEL ESTIMATION (Vectorized + Parallelized - EXACT ndtr)

Processing 34 firms with -1 parallel jobs...

Starting parallel Merton estimation...
Loaded liability data

MERTON MODEL ESTIMATION (Vectorized + Parallelized - EXACT ndtr)

Processing 34 firms with -1 parallel jobs...

Starting parallel Merton estimation...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


...

  Firm 30/34 (gvkey=221616): 3300/3912 dates
  Firm 33/34 (gvkey=241637): 1200/3912 dates
  Firm 31/34 (gvkey=222379): 3200/3912 dates
  Firm 27/34 (gvkey=220833): 3800/3912 dates
  Firm 32/34 (gvkey=241456): 1600/3912 dates
  Firm 34/34 (gvkey=340153): 200/1365 dates
  Firm 33/34 (gvkey=241637): 1300/3912 dates
  Firm 30/34 (gvkey=221616): 3400/3912 dates
  Firm 31/34 (gvkey=222379): 3200/3912 dates
  Firm 27/34 (gvkey=220833): 3800/3912 dates
  Firm 32/34 (gvkey=241456): 1600/3912 dates
  Firm 34/34 (gvkey=340153): 200/1365 dates
  Firm 33/34 (gvkey=241637): 1300/3912 dates
  Firm 30/34 (gvkey=221616): 3400/3912 dates
  Firm 31/34 (gvkey=222379): 3300/3912 dates
  Firm 27/34 (gvkey=220833): 3900/3912 dates
  Firm 32/34 (gvkey=241456): 1700/3912 dates
  Firm 34/34 (gvkey=340153): 300/1365 dates
  Firm 33/34 (gvkey=241637): 1400/3912 dates
  Firm 30/34 (gvkey=221616): 3500/3912 dates
  Firm 31/34 (gvkey=222379): 3300/3912 dates
  Firm 27/34 (gvkey=220833): 3900/3912 dates
  Firm 32/34 

[Parallel(n_jobs=-1)]: Done  31 out of  34 | elapsed:  1.4min remaining:    8.3s


  Firm 33/34 (gvkey=241637): 2000/3912 dates
  Firm 34/34 (gvkey=340153): 800/1365 dates
  Firm 32/34 (gvkey=241456): 2400/3912 dates
  Firm 33/34 (gvkey=241637): 2100/3912 dates
  Firm 32/34 (gvkey=241456): 2400/3912 dates
  Firm 33/34 (gvkey=241637): 2100/3912 dates
  Firm 32/34 (gvkey=241456): 2500/3912 dates
  Firm 34/34 (gvkey=340153): 900/1365 dates
  Firm 33/34 (gvkey=241637): 2200/3912 dates
  Firm 32/34 (gvkey=241456): 2500/3912 dates
  Firm 34/34 (gvkey=340153): 900/1365 dates
  Firm 33/34 (gvkey=241637): 2200/3912 dates
  Firm 32/34 (gvkey=241456): 2600/3912 dates
  Firm 33/34 (gvkey=241637): 2300/3912 dates
  Firm 32/34 (gvkey=241456): 2600/3912 dates
  Firm 33/34 (gvkey=241637): 2300/3912 dates
  Firm 34/34 (gvkey=340153): 1000/1365 dates
  Firm 32/34 (gvkey=241456): 2700/3912 dates
  Firm 34/34 (gvkey=340153): 1000/1365 dates
  Firm 32/34 (gvkey=241456): 2700/3912 dates
  Firm 33/34 (gvkey=241637): 2400/3912 dates
  Firm 33/34 (gvkey=241637): 2400/3912 dates
  Firm 34/34 

[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:  1.5min finished



✓ Parallel Merton complete in 0:01:32

Caching 125,648 Merton results...
✓ Cached to: /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/intermediates/merton_results_cache.pkl

Merton Estimation Complete
Total time: 0:01:32
Firms processed: 34
Daily results: 125,614

Saved to /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/output
Saved to /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/output


### 3. GARCH(1,1) Estimation
Estimates a standard GARCH(1,1) model on the daily asset returns derived from the Merton model.
*   **Input**: `daily_asset_returns.csv`
*   **Output**: `daily_asset_returns_with_garch.csv` (adds conditional volatility columns).

In [4]:
final_daily_returns = run_garch_estimation(daily_returns_df)

# Save Results
final_daily_returns.to_csv(config.OUTPUT_DIR / "daily_asset_returns_with_garch.csv", index=False)
print("Saved 'daily_asset_returns_with_garch.csv'")

Estimating GARCH(1,1) with t-distribution on DAILY Asset Returns...
Processing GARCH for 34 firms (Daily Data)...


Estimating GARCH(1,1) with t-distribution on DAILY Asset Returns...
Processing GARCH for 34 firms (Daily Data)...


Processed GARCH for 10 firms...
Processed GARCH for 20 firms...
Processed GARCH for 20 firms...
Processed GARCH for 30 firms...
Processed GARCH for 30 firms...
GARCH estimation complete.
GARCH estimation complete.
Saved 'daily_asset_returns_with_garch.csv'
Saved 'daily_asset_returns_with_garch.csv'


### 4. Regime-Switching Model (Hamilton Filter)
Estimates a 2-state Markov Switching model on returns (High Volatility / Low Volatility states), independent of GARCH dynamics inside the states.
*   **Output**: `daily_asset_returns_with_regime.csv`.

In [5]:
final_daily_returns_rs = run_regime_switching_estimation(daily_returns_df)

# Save Results
final_daily_returns_rs.to_csv(config.OUTPUT_DIR / "daily_asset_returns_with_regime.csv", index=False)
print("Saved 'daily_asset_returns_with_regime.csv'")

Estimating Regime Switching Model (2-Regime Markov) on DAILY Returns...
(Hamilton Filter)

Processing Regime Switching for 34 firms...

  ✓ Firm 1/34: gvkey=14447
      Observations: 3762
      Regime 0: μ=0.499866, σ=0.008699
      Regime 1: μ=0.499871, σ=0.009390
      Transition: P(0→0)=0.316, P(1→1)=0.688

  ✓ Firm 1/34: gvkey=14447
      Observations: 3762
      Regime 0: μ=0.499866, σ=0.008699
      Regime 1: μ=0.499871, σ=0.009390
      Transition: P(0→0)=0.316, P(1→1)=0.688

  ✓ Firm 2/34: gvkey=15532
      Observations: 3762
      Regime 0: μ=0.499996, σ=0.098713
      Regime 1: μ=0.499997, σ=0.097689
      Transition: P(0→0)=0.501, P(1→1)=0.548

  ✓ Firm 2/34: gvkey=15532
      Observations: 3762
      Regime 0: μ=0.499996, σ=0.098713
      Regime 1: μ=0.499997, σ=0.097689
      Transition: P(0→0)=0.501, P(1→1)=0.548

  ✓ Firm 3/34: gvkey=15549
      Observations: 3762
      Regime 0: μ=0.500000, σ=0.177171
      Regime 1: μ=0.500000, σ=0.180112
      Transition: P(0→0)=0.564

### 5. Optimized MS-GARCH Estimation
Estimates a "True" MS-GARCH model where each regime has its own GARCH(1,1) process.
*   **Optimizations**: Uses Warm Start, JIT Compilation (Numba), and Numerical Optimizations.
*   **Output**: `daily_asset_returns_with_msgarch.csv` and parameter file.

In [6]:
print("Running MS-GARCH Estimation...")
final_daily_returns_msgarch = run_ms_garch_estimation_optimized(
    daily_returns_df,
    output_file=str(config.OUTPUT_DIR / "ms_garch_parameters.csv")
)

# Save Results
final_daily_returns_msgarch.to_csv(config.OUTPUT_DIR / "daily_asset_returns_with_msgarch.csv", index=False)
print("Saved 'daily_asset_returns_with_msgarch.csv'")

Running MS-GARCH Estimation...

OPTIMIZED MS-GARCH ESTIMATION
Optimizations enabled:
  ✓ GARCH(1,1) warm start for initial parameters
  ✓ Numba JIT-compiled Hamilton filter
  ✓ Cached intermediate results
  ✓ L-BFGS-B optimizer with tuned settings

Processing 34 firms...

[1/34] Processing 14447
  Fitting OPTIMIZED MS-GARCH with t-distribution...
    → Getting GARCH(1,1) warm start parameters...
    → Warm start: omega=1.74e-06, alpha=0.043, beta=0.949, nu=4.4
    → Running L-BFGS-B optimization...
  MLE converged: True
  Log-likelihood: 11383.77
  Regime 0 (low vol): omega=0.000001, alpha=0.0412, beta=0.9575, nu=4.56
  Regime 1 (high vol): omega=0.000003, alpha=0.0550, beta=0.9102, nu=4.47
  Persistence: p00=0.9534, p11=0.9523
  MLE converged: True
  Log-likelihood: 11383.77
  Regime 0 (low vol): omega=0.000001, alpha=0.0412, beta=0.9575, nu=4.56
  Regime 1 (high vol): omega=0.000003, alpha=0.0550, beta=0.9102, nu=4.47
  Persistence: p00=0.9534, p11=0.9523
  ✓ Successfully estimated M

### 6. Probability of Default (PD) Calculation
Calculates PD using the Merton Model formula but substituting the volatility estimates from GARCH, RS, and MS-GARCH models.
*   **Benchmark**: Calculates standard Merton PD assuming Normal distribution.
*   **Output**: `daily_pd_results.csv`.

In [7]:
pd_results = run_pd_pipeline(
    str(config.OUTPUT_DIR / 'daily_asset_returns_with_garch.csv'), 
    str(config.OUTPUT_DIR / 'daily_asset_returns_with_regime.csv'), 
    str(config.OUTPUT_DIR / 'daily_asset_returns_with_msgarch.csv')
)

pd_results.to_csv(config.OUTPUT_DIR / "daily_pd_results.csv", index=False)
print("Saved 'daily_pd_results.csv'")

# Merton PD (Benchmark)
merton_normal_pd = calculate_merton_pd_normal(str(config.OUTPUT_DIR / 'daily_asset_returns.csv'))
merton_normal_pd.to_csv(config.OUTPUT_DIR / "daily_pd_results_merton_normal.csv", index=False)
print("Saved 'daily_pd_results_merton_normal.csv'")


PROBABILITY OF DEFAULT CALCULATION (Multi-Model, Daily Data)
Loading auxiliary data (liabilities and interest rates)...
  Loading liabilities...
    ✓ Loaded 741 liability records
  Loading interest rates...
    ✓ Loaded 384 months of interest rate data

Calculating PD for each model...

  Processing GARCH...
    ✓ Loaded 741 liability records
  Loading interest rates...
    ✓ Loaded 384 months of interest rate data

Calculating PD for each model...

  Processing GARCH...
    ✓ GARCH: Calculated PD for 125,614 observations
       PD column name: 'pd_garch'

  Processing Regime Switching...
    ✓ GARCH: Calculated PD for 125,614 observations
       PD column name: 'pd_garch'

  Processing Regime Switching...
    ✗ Volatility column 'garch_volatility' not found in /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/output/daily_asset_returns_with_regime.csv.
       Available columns: ['gvkey', 'date', 'asset_return_daily', 'asset_value', 'asset_volatility', 'regime_state', 'regime

### 7. Monte Carlo Simulation (GARCH) & Diagnostics
Simulates future asset values for 1 year (252 days) using GARCH volatility dynamics.
Also runs **Volatility Diagnostics** to identify firms with explosive volatility that might distort results.
*   **Output**: `daily_monte_carlo_garch_results.csv` and diagnostic files in `data/diagnostics/`.

In [8]:
print("Run Monte Carlo GARCH (1 year) - PARALLEL with all cores...")
mc_results = monte_carlo_garch_1year_parallel(
    str(config.OUTPUT_DIR / 'daily_asset_returns_with_garch.csv'), 
    gvkey_selected=None, 
    num_simulations=1000,
    num_days=252,
    n_jobs=-1  # Use all available cores
)
mc_results.to_csv(config.OUTPUT_DIR / "daily_monte_carlo_garch_results.csv", index=False)

# Diagnostics
print("Running Volatility Diagnostics...")
diagnostics_results = run_volatility_diagnostics(
    garch_file=str(config.OUTPUT_DIR / 'daily_asset_returns_with_garch.csv'),
    mc_garch_file=str(config.OUTPUT_DIR / 'daily_monte_carlo_garch_results.csv'),
    output_dir=str(config.DIAGNOSTICS_DIR)
)

PROBLEMATIC_FIRMS = diagnostics_results['problematic_firms']
CLEAN_FIRMS = diagnostics_results['clean_firms']
print(f"Problematic Firms: {len(PROBLEMATIC_FIRMS)}")

Run Monte Carlo GARCH (1 year) - PARALLEL with all cores...
Loading GARCH data from /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/output/daily_asset_returns_with_garch.csv...
Running PARALLELIZED Monte Carlo GARCH simulation:
  Firms: 34
  Dates: 4015
  Simulations per firm: 1,000
  Forecast horizon: 252 days
  Parallel jobs: -1
  Innovation distribution: Student's t
Running PARALLELIZED Monte Carlo GARCH simulation:
  Firms: 34
  Dates: 4015
  Simulations per firm: 1,000
  Forecast horizon: 252 days
  Parallel jobs: -1
  Innovation distribution: Student's t

Processing 4015 dates in parallel...

Processing 4015 dates in parallel...


Run Monte Carlo GARCH (1 year) - PARALLEL with all cores...
Loading GARCH data from /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/output/daily_asset_returns_with_garch.csv...
Running PARALLELIZED Monte Carlo GARCH simulation:
  Firms: 34
  Dates: 4015
  Simulations per firm: 1,000
  Forecast horizon: 252 days
  Parallel jobs: -1
  Innovation distribution: Student's t
Running PARALLELIZED Monte Carlo GARCH simulation:
  Firms: 34
  Dates: 4015
  Simulations per firm: 1,000
  Forecast horizon: 252 days
  Parallel jobs: -1
  Innovation distribution: Student's t

Processing 4015 dates in parallel...

Processing 4015 dates in parallel...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   

Run Monte Carlo GARCH (1 year) - PARALLEL with all cores...
Loading GARCH data from /Users/thomasdeleeuw/Downloads/Seminar-QF/Seminar QF/data/output/daily_asset_returns_with_garch.csv...
Running PARALLELIZED Monte Carlo GARCH simulation:
  Firms: 34
  Dates: 4015
  Simulations per firm: 1,000
  Forecast horizon: 252 days
  Parallel jobs: -1
  Innovation distribution: Student's t
Running PARALLELIZED Monte Carlo GARCH simulation:
  Firms: 34
  Dates: 4015
  Simulations per firm: 1,000
  Forecast horizon: 252 days
  Parallel jobs: -1
  Innovation distribution: Student's t

Processing 4015 dates in parallel...

Processing 4015 dates in parallel...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   


PARALLELIZED MONTE CARLO GARCH COMPLETE
Total time: 0:29:16
Rows per second: 72
Speedup: ~-1x expected on -1-core system

Running Volatility Diagnostics...

VOLATILITY DIAGNOSTICS: IDENTIFYING PROBLEMATIC FIRMS

Running Volatility Diagnostics...

VOLATILITY DIAGNOSTICS: IDENTIFYING PROBLEMATIC FIRMS

✓ Loaded GARCH data: 125,614 observations
✓ Number of firms: 34

------------------------------------------------------------
PART 1: GARCH PARAMETER ANALYSIS
------------------------------------------------------------


------------------------------------------------------------
PART 2: MONTE CARLO VOLATILITY ANALYSIS
------------------------------------------------------------

✓ Loaded GARCH data: 125,614 observations
✓ Number of firms: 34

------------------------------------------------------------
PART 1: GARCH PARAMETER ANALYSIS
------------------------------------------------------------


------------------------------------------------------------
PART 2: MONTE CARLO VOLATILIT

### 8. Monte Carlo Simulation (Regime Switching & MS-GARCH)
Runs MC simulations for the Regime-Switching and MS-GARCH models.
*   **Regime Switching**: Simulates regime changes and draws returns based on state-specific volatility (no GARCH).
*   **MS-GARCH**: Simulates regime changes AND GARCH dynamics within each state.

In [9]:
print("Run MC Regime Switching - PARALLEL with all cores...")
mc_rs_results = monte_carlo_regime_switching_1year_parallel(
    garch_file=str(config.OUTPUT_DIR / 'daily_asset_returns_with_garch.csv'),
    regime_params_file=str(config.OUTPUT_DIR / 'regime_switching_parameters.csv'),
    gvkey_selected=None,
    num_simulations=1000,
    num_days=252,
    n_jobs=-1  # Use all available cores
)
mc_rs_results.to_csv(config.OUTPUT_DIR / "daily_monte_carlo_regime_switching_results.csv", index=False)

print("Run MC MS-GARCH - PARALLEL with all cores...")
mc_msgarch_results = monte_carlo_ms_garch_1year_parallel(
    daily_returns_file=str(config.OUTPUT_DIR / 'daily_asset_returns_with_msgarch.csv'),
    ms_garch_params_file=str(config.OUTPUT_DIR / 'ms_garch_parameters.csv'),
    gvkey_selected=None,
    num_simulations=1000,
    num_days=252,
    n_jobs=-1  # Use all available cores
)
mc_msgarch_results.to_csv(config.OUTPUT_DIR / "daily_monte_carlo_ms_garch_results.csv", index=False)

[Parallel(n_jobs=-1)]: Done 2649 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 2722 tasks      | elapsed: 28.3min
[Parallel(n_jobs=-1)]: Done 2797 tasks      | elapsed: 29.0min
[Parallel(n_jobs=-1)]: Done 2872 tasks      | elapsed: 29.7min
[Parallel(n_jobs=-1)]: Done 2949 tasks      | elapsed: 30.5min
[Parallel(n_jobs=-1)]: Done 3026 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done 3105 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 32.7min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 33.5min
[Parallel(n_jobs=-1)]: Done 3346 tasks      | elapsed: 34.3min
[Parallel(n_jobs=-1)]: Done 3429 tasks      | elapsed: 35.0min
[Parallel(n_jobs=-1)]: Done 3512 tasks      | elapsed: 35.8min
[Parallel(n_jobs=-1)]: Done 3597 tasks      | elapsed: 36.6min
[Parallel(n_jobs=-1)]: Done 3682 tasks      | elapsed: 37.4min
[Parallel(n_jobs=-1)]: Done 3769 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 3856 tasks      | elapsed: 


Regime Fraction Statistics:
  Regime 0 (low vol): mean=0.510
  Regime 1 (high vol): mean=0.490

PARALLELIZED MONTE CARLO MS-GARCH COMPLETE
Total time: 0:38:18
Rows per second: 55
Speedup: ~-1x expected on -1-core system



### 9. CDS Spread Calculation
Calculates Model-Implied CDS Spreads for 1, 3, and 5-year horizons.
Input: Monte Carlo results from all 3 models.
Output: Separate CSV files for CDS spreads (cleaned of problematic firms).

In [15]:
import importlib
from src.analysis import cds_spread_calculator
importlib.reload(cds_spread_calculator)
from src.analysis.cds_spread_calculator import CDSSpreadCalculator

cds_calc = CDSSpreadCalculator(maturity_horizons=[1, 3, 5])

# --- 1. Merton Normal (Analytical) ---
print("CDS Spreads: Merton Normal (Analytical)")
merton_file = config.OUTPUT_DIR / 'merged_data_with_merton.csv'
if merton_file.exists():
    df_cds_spreads_merton = cds_calc.calculate_cds_spreads_analytical_merton(
        merton_file=str(merton_file),
        output_file=str(config.OUTPUT_DIR / 'cds_spreads_merton.csv')
    )
else:
    print(f"Skipping Merton: Input file {merton_file} not found.")

# --- 2. GARCH ---
print("\nCDS Spreads: GARCH")
mc_garch_file = config.OUTPUT_DIR / 'daily_monte_carlo_garch_results.csv'
if mc_garch_file.exists():
    df_cds_spreads_garch_all = cds_calc.calculate_cds_spreads_from_mc_garch(
        mc_garch_file=str(mc_garch_file),
        daily_returns_file=str(config.OUTPUT_DIR / 'daily_asset_returns.csv'),
        merton_file=str(config.OUTPUT_DIR / 'merged_data_with_merton.csv'),
        output_file=str(config.OUTPUT_DIR / 'cds_spreads_garch_mc_all_firms.csv')
    )
else:
    print(f"Skipping GARCH: Input file {mc_garch_file} not found. (Run MC GARCH cell first)")

# --- 3. Regime Switching ---
print("\nCDS Spreads: RS")
mc_rs_file = config.OUTPUT_DIR / 'daily_monte_carlo_regime_switching_results.csv'
if mc_rs_file.exists():
    df_cds_spreads_rs_all = cds_calc.calculate_cds_spreads_from_mc_garch(
        mc_garch_file=str(mc_rs_file),
        daily_returns_file=str(config.OUTPUT_DIR / 'daily_asset_returns.csv'),
        merton_file=str(config.OUTPUT_DIR / 'merged_data_with_merton.csv'),
        output_file=str(config.OUTPUT_DIR / 'cds_spreads_regime_switching_mc_all_firms.csv')
    )
else:
    print(f"Skipping RS: Input file {mc_rs_file} not found. (Run MC Regime Switching cell first)")

# --- 4. MS-GARCH ---
print("\nCDS Spreads: MS-GARCH")
mc_msgarch_file = config.OUTPUT_DIR / 'daily_monte_carlo_ms_garch_results.csv'
if mc_msgarch_file.exists():
    df_cds_spreads_msgarch_all = cds_calc.calculate_cds_spreads_from_mc_garch(
        mc_garch_file=str(mc_msgarch_file),
        daily_returns_file=str(config.OUTPUT_DIR / 'daily_asset_returns.csv'),
        merton_file=str(config.OUTPUT_DIR / 'merged_data_with_merton.csv'),
        output_file=str(config.OUTPUT_DIR / 'cds_spreads_ms_garch_mc_all_firms.csv'),
        volatility_column='mc_msgarch_cumulative_volatility'
    )
else:
    print(f"Skipping MS-GARCH: Input file {mc_msgarch_file} not found. (Run MC MS-GARCH cell first)")


CDS Spread Calculator initialized (Section 2.4.2):
  CDS Maturities: [1, 3, 5] years
  Methodology: Malone et al. (2009) - Risk-neutral valuation

CDS Spreads: Merton Normal (Analytical)

ANALYTICAL MERTON CDS SPREADS

Loading Merton results from C:\Users\Chase\Downloads\Seminar QF\Seminar QF\data\output\merged_data_with_merton.csv...
Filtered 15432 rows with missing values.
Loading Interest Rates...
Merging Interest Rates...
Calculating CDS Spreads (Analytical Merton)...
Saving results to C:\Users\Chase\Downloads\Seminar QF\Seminar QF\data\output\cds_spreads_merton.csv...
Done calculation.


CDS Spreads: GARCH
Skipping GARCH: Input file C:\Users\Chase\Downloads\Seminar QF\Seminar QF\data\output\daily_monte_carlo_garch_results.csv not found. (Run MC GARCH cell first)

CDS Spreads: RS
Skipping RS: Input file C:\Users\Chase\Downloads\Seminar QF\Seminar QF\data\output\daily_monte_carlo_regime_switching_results.csv not found. (Run MC Regime Switching cell first)

CDS Spreads: MS-GARCH
Ski

### 10. Model vs Market CDS Spread Correlation
Compare model-implied CDS spreads with actual market CDS data.
- Loads real CDS data (in basis points) for 1Y, 3Y, and 5Y maturities.
- Matches companies between model and market data.
- Calculates correlations for each firm and overall.

In [16]:
# Reload the module to pick up changes
import importlib
from src.analysis import cds_correlation
importlib.reload(cds_correlation)
from src.analysis.cds_correlation import run_cds_correlation_analysis, plot_cds_correlations

# Run CDS Correlation Analysis
print("Running CDS Correlation Analysis...")
correlation_results = run_cds_correlation_analysis()

# Plot scatter plots (5-year maturity)
plot_cds_correlations(correlation_results, maturity=5)

Running CDS Correlation Analysis...
Loading real CDS market data...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Chase\\Downloads\\Seminar QF\\Seminar QF\\data\\input\\CDS_1y_mat_data.xlsx'